### 4.	모델을 만들기 위한 데이터 전처리 작업

In [2]:
### 불용어 제거

In [3]:
### 정수 인코딩

In [4]:
### 라벨링 one hot encoding 적용

### 5.	모델 만들기


In [5]:
### 전체 데이터 길이를 동일하게 맞추기 (pad sequence)

In [6]:
### LSTM, BERT로 1차적으로 모델 생성하기

In [1]:
!pip install pytorch-transformers

You should consider upgrading via the 'c:\users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [2]:
path='C:/Users/User/Desktop/논문준비/'

In [35]:
train_df= pd.read_csv(path+'ratings_train.txt', sep='\t')
test_df= pd.read_csv(path+'ratings_test.txt', sep='\t')

In [36]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [38]:
train_df= train_df[:1000]
train_df

,id,document,label
103096,1269448,이 영화(제작과정포함)를 접한 후 결론 → 샤론스톤은 쓰레기다.,0
4038,9867479,넘 멋져열. 몇번씩 보게되는~~,1
135345,9924513,호러 공포영화 정말 좋아해서 거의 다 봤어요. 최근 본 공포물중 최고입니다. 쫄깃쫄...,1
119472,7074014,웃으면서 잘 봤네요,1
64192,8359338,어음...이거 다 보고 딱 생각난대사가 ...뭐지...? 왜 이렇게 갑자기 끝나? ...,0
...,...,...,...
97070,8748577,이런 다방이 있으면 매일 출근도장 찍을텐데. 이런 영화를 만들다니 매일 욕먹어도 모자랄듯,0
49141,8264721,책이 확실히 더 재밌긴 하네요 로건 레먼 보고 안구정화 했으니10점!!♡,1
74847,418346,어린 소녀가 극장표를 직접 끊고 봤던 영화.성룡액션최고다,1
91599,5720945,손범수씨가 진행하던 동물의 왕국이 그립네요.,0


In [39]:
train_df['id']

103096    1269448
4038      9867479
135345    9924513
119472    7074014
64192     8359338
           ...   
97070     8748577
49141     8264721
74847      418346
91599     5720945
107898    8000217
Name: id, Length: 1000, dtype: int64

In [40]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [41]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [42]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 1.3 GB
Cached:    1.4 GB


C:\Users\User\anaconda3\lib\site-packages\torch\cuda\memory.py:344: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [43]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

In [44]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [45]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

print(1)

model.train()

print(2)

1
2


In [46]:
import gc

gc.collect()

409

In [ ]:
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        print(1)
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1